In [1]:
using GamsStructure

In [3]:
GU = GamsUniverse()
@GamsSets(GU,"test_data",begin
    :i,""
    :j,""
end)

@load_parameters!(GU,"test_data",begin
    :p, (:i,:j), description => "Hi"
end)

Description: Hi
Domain: (:i, :j)

[1.0 2.0; 3.0 4.0]

In [4]:
GU

Sets

j => 
i => 

Parameters

p => (:i, :j) => Hi

Scalars



In [ ]:
using MacroTools

In [ ]:
macro _load_parameters!(GU,base_path,block)
    GU = esc(GU)
    base_path = esc(base_path)
    if !(isa(block,Expr) && block.head == :block)
        error()
    end

    code = quote end
    for it in block.args
        if isexpr(it,:tuple)
            parm_name = it.args[1]
            domain = it.args[2]
            path = :("$($base_path)/$($parm_name).csv")
            desc = ""
            columns = missing
            value_name = QuoteNode(:value)
            for elm in it.args[3:end]
                if elm.args[1] != :(=>)
                    continue
                end
                if elm.args[2] == :description
                    desc = elm.args[3]
                elseif elm.args[2] == :columns
                    columns = elm.args[3]
                elseif elm.args[2] == :value_name
                    value_name = esc(elm.args[3])
                elseif elm.args[2] == :file_name
                    path = :("$($base_path)/$($(elm.args[3]))")
                end
            end
            push!(code.args, :($load_parameter!($GU,$path,$parm_name,$domain;
                                                description = $desc,
                                                columns = $columns,
                                                value_name = $value_name
                                                )))

        end
    end
    return code
end

In [ ]:
@_load_parameters!(GU,"test_data",begin
    :p, (:i,:j),description => "This is a test",file_name =>"p copy.csv", value_name => :data,columns=>[1,2]
end)

In [ ]:
GU

In [ ]:
@macroexpand(@_load_parameters!(GU,"test_data",begin
:p, (:i,:j)
end))

In [ ]:
function tt(a,d;b=1,c=2)

    return (a+b+c)*d
end

macro testing(block)
    it = block.args[2]
    
    #return(it)
    a = it.args[1]
    d = it.args[2]
    b = 0
    c = 0
    for elm in it.args[3:end]
        if elm.args[2] ==:b
            b = elm.args[3]
        elseif elm.args[2] == :c
            c = elm.args[3]
        end
    end
    #kwargs = ()
    #num = it.args[1]
    #if length(it.args)>1
    #    kwargs = esc(it.args[2])
    #end
    return :($tt($a,$d;b=$b,c=$c))
end

In [ ]:
@testing(begin
    10, 11,c=>8
end)

In [ ]:
@macroexpand @testing begin 10, 11, :b=>2, :c=>3 end

In [ ]:
m = (b=2,c=4)

In [ ]:
b = get(m,:d,0)